In [145]:
def parse(input):
  return [l.strip() for l in open(input, 'r').readlines()]

In [146]:
import re
seeds_rx = r'^seeds: (.*)$'
map_rx = r'^(.*)-to-(.*) map:$'

def process(input):
  maps = []
  state = 'init'

  for line in input:
    if state == 'init':
      seeds = list(map(int, re.match(seeds_rx, line).group(1).split()))
      state = 'after_seeds'
    elif state == 'after_seeds':
      state = 'map'
    elif state == 'map':
      current_map = re.match(map_rx, line).groups()
      current_mappings = []
      state = 'mapping'
    elif state == 'mapping' and len(line) > 0:
      current_mappings.append(tuple(map(int, line.split())))
    else:
      maps.append((current_map, current_mappings))
      state = 'map'
  maps.append((current_map, current_mappings))
  return (seeds, maps)

In [147]:
def first_part(input):
  seeds, maps = process(input)
  locations = []

  for x in seeds:
    for _, mappings in maps:
      for (a, b, l) in mappings:
        if x >= b and x < b + l:
          x += a - b
          break
    locations.append(x)
    
  return min(locations)

assert first_part(parse('sample')) == 35
print(first_part(parse('input')))

51580674


In [148]:
def subtract(a, b):
  before = a[0], min(b[0], a[1])
  after = max(b[1], a[0]), a[1]
  if before[1] > before[0]:
    yield before
  if after[1] > after[0]:
    yield after

assert list(subtract((0, 10), (3, 5))) == [(0, 3), (5, 10)]
assert list(subtract((10, 20), (3, 5))) == [(10, 20)]
assert list(subtract((10, 20), (30, 35))) == [(10, 20)]

def subtract_all(a, l):
  result = [a]
  for b in l:
    r = []
    for a in result:
      r += list(subtract(a, b))
    result = r
  return result

assert list(subtract_all((0, 10), [(3, 5), (7, 8)])) == [(0, 3), (5, 7), (8, 10)]

def intersect(a, b):
  start, end = max(a[0], b[0]), min(a[1], b[1])
  return (start, end) if start < end else None

assert intersect((0, 10), (3, 5)) == (3, 5)
assert intersect((10, 20), (3, 5)) == None
assert intersect((10, 20), (30, 35)) == None

In [149]:
def second_part(input):
  seeds, maps = process(input)
  ranges = list(map(lambda i: (i[0], i[0] + i[1]), zip(seeds[0::2], seeds[1::2])))
  maps = [list(map(lambda m: (m[1], m[1] + m[2], m[0] - m[1]), m[1])) for m in maps]

  for each_map in maps:
    all_new_ranges = []
    for range in ranges:
      ranges_to_subtract = []
      for mapping in each_map:
        if mapped_range := intersect(range, mapping):
          ranges_to_subtract += [mapped_range]
          all_new_ranges += [tuple(map(lambda x: x + mapping[2], mapped_range))]
      all_new_ranges += list(subtract_all(range, ranges_to_subtract))
    ranges = all_new_ranges
  return min(map(lambda r: r[0], ranges))

assert second_part(parse('sample')) == 46
print(second_part(parse('input')))

99751240
